In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql

SELECT 
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [39]:
%%sql

WITH tracks AS
(
    SELECT *
    FROM track t
    JOIN invoice_line il
    ON il.track_id = t.track_id
)

SELECT
    g.name Genre,
    SUM(t.quantity) QuantitySold,
    CAST(SUM(t.quantity) AS FLOAT) / (SELECT CAST(SUM(quantity) AS FLOAT) FROM tracks) Total
FROM genre g
JOIN tracks t
ON g.genre_id = t.genre_id
GROUP BY g.name
ORDER BY SUM(t.quantity) DESC

Done.


Genre,QuantitySold,Total
Rock,2635,0.553920538154299
Metal,619,0.13012402774858103
Alternative & Punk,492,0.10342652932520496
Latin,167,0.03510615934412445
R&B/Soul,159,0.033424427159974776
Blues,124,0.026066848854319948
Jazz,121,0.025436199285263823
Alternative,117,0.024595333193188986
Easy Listening,74,0.015556022703384485
Pop,63,0.013243640950178683


Don't recommend hip-hop because this genre, historically in our records, has the lowest volume of tracks sold.

In [46]:
%%sql

WITH customer_invoice AS
(
    SELECT *
    FROM customer c
    JOIN invoice i
    ON i.customer_id = c.customer_id
    JOIN invoice_line il
    ON il.invoice_id = i.invoice_id
)

SELECT
    e.first_name,
    e.last_name,
    e.title,
    e.hire_date,
    SUM(ci.total)
FROM employee e
LEFT JOIN customer_invoice ci
ON ci.support_rep_id = e.employee_id
WHERE title LIKE '%Support%'
GROUP BY e.last_name

Done.


first_name,last_name,title,hire_date,SUM(ci.total)
Steve,Johnson,Sales Support Agent,2017-10-17 00:00:00,13333.319999999945
Margaret,Park,Sales Support Agent,2017-05-03 00:00:00,15923.159999999847
Jane,Peacock,Sales Support Agent,2017-04-01 00:00:00,18246.689999999773


There is a direct correlation between the hire date and sales made by each sales agent as expected. The earlier their hire date is, the higher the sales figure they accumulate.

In [87]:
%%sql

WITH one_customer AS
(
    SELECT country
    FROM
        (
            SELECT
                country,
                COUNT(customer_id) count
            FROM customer
            GROUP BY country
        )
    WHERE count = 1
)

SELECT
    CASE 
        WHEN c.country IN one_customer
        THEN 'Other'
        ELSE c.country
        END AS Country,
    COUNT(DISTINCT(c.customer_id)) AS NumberofCustomers,
    SUM(i.total) AS TotalSales,
    CAST(SUM(i.total) AS FLOAT) / CAST(COUNT(DISTINCT(c.customer_id)) AS FLOAT) AS 'Customer_Lifetime_Value',
    AVG(i.total) 'Average Order Value'
FROM customer c
JOIN invoice i
ON c.customer_id = i.customer_id
GROUP BY 1
ORDER BY 3 DESC

Done.


Country,NumberofCustomers,TotalSales,Customer_Lifetime_Value,Average Order Value
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857


Top 3 sales by country are USA, Canada, and Brazil.

In [ ]:
%%sql

SELECT
    Category,
    COUNT(DISTINCT(invoice_id)) NumberOfInvoices,
    COUNT(DISTINCT(invoice_id) / (SELECT COUNT(DISTINCT(invoice_id))) Percent
FROM
(
    SELECT a.
    FROM album a
    JOIN track t
    ON t.album_id ON a.album_id
    JOIN invoice_line il
    ON il.track_id = t.track_id
)
GROUP BY 1

In [96]:
%%sql

    SELECT il.invoice_id, il.track_id, t.album_id
    FROM track t
    JOIN invoice_line il
    ON il.track_id = t.track_id
    LIMIT 20

Done.


invoice_id,track_id,album_id
1,1158,91
1,1159,91
1,1160,91
1,1161,91
1,1162,91
1,1163,91
1,1164,91
1,1165,91
1,1166,91
1,1167,91
